In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 25

BASE_PATH = "./archive/brisc2025/segmentation_task"

TRAIN_IMG = f"{BASE_PATH}/train/images"
TRAIN_MASK = f"{BASE_PATH}/train/masks"

TEST_IMG = f"{BASE_PATH}/test/images"
TEST_MASK = f"{BASE_PATH}/test/masks"

PRED_BASE = f"{BASE_PATH}/predictions/test"

PRED_MASK_DIR = f"{PRED_BASE}/masks"
OVERLAY_DIR = f"{PRED_BASE}/overlays"

os.makedirs(PRED_MASK_DIR, exist_ok=True)
os.makedirs(OVERLAY_DIR, exist_ok=True)

In [5]:
def load_image_mask(img_path, mask_path):

    #Load image 
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = img_to_array(img).astype("float32") / 255.0

    #Load mask (grayscale)
    mask = load_img(
        mask_path,
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="grayscale"
    )
    mask = img_to_array(mask).astype("float32") / 255.0

    #Binarize mask 
    mask = (mask > 0.5).astype("float32")

    return img, mask


In [6]:
def data_generator(img_dir, mask_dir, batch_size):

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    while True:
        np.random.shuffle(img_files)

        for i in range(0, len(img_files), batch_size):

            batch_files = img_files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch_files:

                img_path = os.path.join(img_dir, img_file)

                # mask name must match image stem
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    print(f"Missing mask: {mask_file}")
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)


In [7]:
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    
    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    return x

In [8]:
def unet(input_shape=(IMG_SIZE, IMG_SIZE, 3)):

    inputs = layers.Input(input_shape)

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D()(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D()(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D()(c3)

    # Bottleneck
    bn = conv_block(p3, 512)

    # Decoder
    u4 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(bn)
    u4 = layers.Concatenate()([u4, c3])
    c4 = conv_block(u4, 256)

    u5 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(c4)
    u5 = layers.Concatenate()([u5, c2])
    c5 = conv_block(u5, 128)

    u6 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(c5)
    u6 = layers.Concatenate()([u6, c1])
    c6 = conv_block(u6, 64)

    outputs = layers.Conv2D(1, 1, activation="sigmoid")(c6)

    return models.Model(inputs, outputs)

In [9]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred):
    smooth = 1e-6

    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth
    )


def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

In [10]:
model = unet()

model.compile(
    optimizer=Adam(1e-4),
    loss=bce_dice_loss,
    metrics=["accuracy", dice_coefficient]
)

model.summary()

I0000 00:00:1770101897.294349  102438 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4132 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 7,708,609 (29.41 MB)

 Trainable params: 7,702,977 (29.38 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))


2026-02-03 12:27:33.867278: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-03 12:27:34.049742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-03 12:27:35.018463: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version: 2.20.0
GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
from sklearn.model_selection import train_test_split

# Split TRAIN into Train / Val

all_train_files = sorted([
    f for f in os.listdir(TRAIN_IMG)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

train_files, val_files = train_test_split(
    all_train_files,
    test_size=0.2,
    random_state=42
)

print("Train samples:", len(train_files))
print("Val samples:", len(val_files))


# Generators from subsets

def subset_generator(img_dir, mask_dir, files, batch_size):

    while True:
        np.random.shuffle(files)

        for i in range(0, len(files), batch_size):

            batch = files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch:

                img_path = os.path.join(img_dir, img_file)
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)


train_gen = subset_generator(
    TRAIN_IMG,
    TRAIN_MASK,
    train_files,
    BATCH_SIZE
)

val_gen = subset_generator(
    TRAIN_IMG,
    TRAIN_MASK,
    val_files,
    BATCH_SIZE
)


# Fit Model
model.fit(
    train_gen,
    steps_per_epoch=len(train_files) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(val_files) // BATCH_SIZE,
    epochs=EPOCHS
)

Train samples: 3146
Val samples: 787
Epoch 1/25


2026-02-03 12:28:29.274343: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fd028017d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-03 12:28:29.274360: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-02-03 12:28:29.379040: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-03 12:28:30.194061: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91801
2026-02-03 12:28:31.515040: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-02-03 12:28:31.96021

393/393 ━━━━━━━━━━━━━━━━━━━━ 163s 331ms/step - accuracy: 0.9829 - dice_coefficient: 0.2839 - loss: 0.8089 - val_accuracy: 0.9839 - val_dice_coefficient: 0.1793 - val_loss: 0.8988
Epoch 2/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 142s 331ms/step - accuracy: 0.9908 - dice_coefficient: 0.5391 - loss: 0.5047 - val_accuracy: 0.9908 - val_dice_coefficient: 0.5708 - val_loss: 0.4729
Epoch 3/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 133s 337ms/step - accuracy: 0.9923 - dice_coefficient: 0.6625 - loss: 0.3741 - val_accuracy: 0.9913 - val_dice_coefficient: 0.6258 - val_loss: 0.4136
Epoch 4/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 128s 327ms/step - accuracy: 0.9928 - dice_coefficient: 0.7132 - loss: 0.3219 - val_accuracy: 0.9907 - val_dice_coefficient: 0.6297 - val_loss: 0.4146
Epoch 5/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 129s 327ms/step - accuracy: 0.9936 - dice_coefficient: 0.7560 - loss: 0.2758 - val_accuracy: 0.9897 - val_dice_coefficient: 0.6041 - val_loss: 0.4483
Epoch 6/25
393/393 ━━━━━━━━━━━━━━━━━━━━ 129s 327ms/step - acc

In [13]:
def predict_and_save(model, img_dir, mask_dir):

    os.makedirs(PRED_MASK_DIR, exist_ok=True)
    os.makedirs(OVERLAY_DIR, exist_ok=True)

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    for file in img_files:

        # image path
        img_path = os.path.join(img_dir, file)

        # mask name
        mask_file = os.path.splitext(file)[0] + ".png"
        mask_path = os.path.join(mask_dir, mask_file)

        # skip if mask missing 
        if not os.path.exists(mask_path):
            print(f"Skipping (mask not found): {file}")
            continue

        # load image & mask 
        img, true_mask = load_image_mask(img_path, mask_path)

        # prediction 
        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_mask = (pred > 0.5).astype(np.uint8) * 255

        # save predicted mask 
        cv2.imwrite(
            os.path.join(PRED_MASK_DIR, mask_file),
            pred_mask.squeeze()
        )

        # overlay 
        img_vis = cv2.imread(img_path)
        img_vis = cv2.resize(img_vis, (IMG_SIZE, IMG_SIZE))

        overlay = img_vis.copy()
        overlay[pred_mask.squeeze() == 255] = [0, 0, 255]  # red tumor

        combined = cv2.addWeighted(img_vis, 0.7, overlay, 0.3, 0)

        cv2.imwrite(
            os.path.join(OVERLAY_DIR, file),
            combined
        )

    print("Predictions, masks & overlays saved successfully.")


In [14]:
predict_and_save(model, TEST_IMG, TEST_MASK)

Predictions, masks & overlays saved successfully.


In [17]:
model.save("brisc2025_unet_latest.keras")

In [18]:
from tensorflow.keras.models import load_model

model = load_model(
    "brisc2025_unet_latest.keras",
    custom_objects={
        "bce_dice_loss": bce_dice_loss,
        "dice_coefficient": dice_coefficient
    }
)

In [21]:
from tqdm import tqdm
import os
import numpy as np

# Metric helpers (NumPy)

def dice_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)

    return (2. * intersection + smooth) / (
        np.sum(y_true) + np.sum(y_pred) + smooth
    )


def iou_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection

    return (intersection + smooth) / (union + smooth)


def precision_recall_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)

    return precision, recall


def accuracy_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    correct = np.sum(y_true == y_pred)
    total = len(y_true)

    return correct / total


# Evaluate on test set

def evaluate_test_set(model, img_dir, mask_dir):

    image_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    dice_scores = []
    iou_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    print(" Evaluating test set...")

    for file in tqdm(image_files):

        img_path = os.path.join(img_dir, file)

        mask_path = os.path.join(
            mask_dir,
            os.path.splitext(file)[0] + ".png"
        )

        if not os.path.exists(mask_path):
            continue

        img, gt_mask = load_image_mask(img_path, mask_path)

        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_bin = (pred > 0.5).astype(np.uint8)

        gt_bin = gt_mask.astype(np.uint8)

        dice_scores.append(dice_np(gt_bin, pred_bin))
        iou_scores.append(iou_np(gt_bin, pred_bin))

        p, r = precision_recall_np(gt_bin, pred_bin)
        precision_scores.append(p)
        recall_scores.append(r)

        accuracy_scores.append(
            accuracy_np(gt_bin, pred_bin)
        )

    print("\n===== TEST SET RESULTS =====")
    print(f"Mean Dice     : {np.mean(dice_scores):.4f}")
    print(f"Mean IoU      : {np.mean(iou_scores):.4f}")
    print(f"Mean Precision: {np.mean(precision_scores):.4f}")
    print(f"Mean Recall   : {np.mean(recall_scores):.4f}")
    print(f"Mean Accuracy : {np.mean(accuracy_scores):.4f}")

    return {
        "dice": dice_scores,
        "iou": iou_scores,
        "precision": precision_scores,
        "recall": recall_scores,
        "accuracy": accuracy_scores,
    }


# RUN EVALUATION
results = evaluate_test_set(
    model,
    TEST_IMG,
    TEST_MASK
)


 Evaluating test set...


100%|████████████████████████████████████████████████████████████████████████████████████████| 860/860 [00:52<00:00, 16.53it/s]


📊 ===== TEST SET RESULTS =====
Mean Dice     : 0.8080
Mean IoU      : 0.7341
Mean Precision: 0.8566
Mean Recall   : 0.8063
Mean Accuracy : 0.9949
